In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
df = pd.read_parquet("../data/sc_pre_processado.parquet")

In [ ]:
# Gambiarra para funcionar (deve ser passado para o pre_processamnto)
df['idade_calculada'] = (
    df['idade_calculada']
    .mask((df['idade_calculada'] < 0) | (df['idade_calculada'] > 120))
    .fillna(df['paciente_idade'])
    .clip(lower=0, upper=120)
)

df = df[df['faixa_etaria'].notnull()]

In [ ]:
# Distribuicao por sexo e faixa etaria
plt.figure(figsize=(8,4))
sns.countplot(x='paciente_enumSexoBiologico', data=df)
plt.title("Distribuição por Sexo")
plt.show()

plt.figure(figsize=(10,5))
sns.countplot(x='faixa_etaria', data=df, order=df['faixa_etaria'].cat.categories)
plt.title("Distribuição por Faixa Etária")
plt.show()

In [ ]:
# Evolucao temporal das aplicacoes de vacina
aplicacoes_tempo = df.groupby('vacina_dataAplicacao').size().rolling(7).mean()
plt.figure(figsize=(12,5))
aplicacoes_tempo.plot()
plt.title("Média móvel (7 dias) de aplicações de vacina em SC")
plt.xlabel("Data de aplicação")
plt.ylabel("Nº de doses")
plt.show()

In [ ]:
# Top 10 municipios com mais doses aplicadas
municipios = df['paciente_endereco_nmMunicipio'].value_counts().head(10)
plt.figure(figsize=(10,6))
sns.barplot(x=municipios.values, y=municipios.index)
plt.title("Top 10 Municípios com mais doses aplicadas")
plt.xlabel("Doses aplicadas")
plt.ylabel("Município")
plt.show()

In [ ]:
# Fabricante mais utilizado ao longo do tempo
fabricante_tempo = df.groupby(['ano_aplicacao', 'vacina_fabricante_nome']).size().reset_index(name='doses')
plt.figure(figsize=(12,6))
sns.lineplot(data=fabricante_tempo, x='ano_aplicacao', y='doses', hue='vacina_fabricante_nome')
plt.title("Evolução anual por fabricante de vacina")
plt.show()

In [ ]:
# Correlacao entre idade e tipo de dose
plt.figure(figsize=(10,6))
sns.boxplot(x='vacina_descricao_dose', y='idade_calculada', data=df)
plt.title("Distribuição de idade por tipo de dose")
plt.show()

In [ ]:
# Clusterizacao por perfil de vacinacao com KMEANS
# Selecionar features relevantes
features = df[['idade_calculada', 'ano_aplicacao']]
features = features.dropna()
scaled = StandardScaler().fit_transform(features)

kmeans = KMeans(n_clusters=4, random_state=42)
df.loc[features.index, 'cluster'] = kmeans.fit_predict(scaled)

sns.scatterplot(x='idade_calculada', y='ano_aplicacao', hue='cluster', data=df)
plt.title("Clusterização de perfis de vacinação")
plt.show()